In [42]:
import cv2 
images_path=r'images/'
import os
import numpy as np
import glob

In [43]:
orb = cv2.ORB_create(3000)
FLANN_INDEX_LSH = 6
index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(indexParams=index_params, searchParams=search_params)

In [44]:
def get_matches(prev_img,current_img):
        """
        This function detect and compute keypoints and descriptors from the i-1'th and i'th image using the class orb object

        Parameters
        ----------
        i (int): The current frame

        Returns
        -------
        q1 (ndarray): The good keypoints matches position in i-1'th image
        q2 (ndarray): The good keypoints matches position in i'th image
        """
        # Find the keypoints and descriptors with ORB
        prev_img_kp, prev_img_des = orb.detectAndCompute(prev_img, None)
        curr_img_kp, curr_img_des = orb.detectAndCompute(current_img, None)
        # Find matches
        matches = flann.knnMatch(prev_img_des,curr_img_des, k=2)

        # Find the matches there do not have a to high distance
        good = []
        try:
            for m, n in matches:
                if m.distance < 0.8 * n.distance:
                    good.append(m)
        except ValueError:
            pass

        draw_params = dict(matchColor = -1, # draw matches in green color
                 singlePointColor = None,
                 matchesMask = None, # draw only inliers
                 flags = 2)

        img3 = cv2.drawMatches(current_img,prev_img_kp,prev_img,curr_img_kp, good ,None,**draw_params)
        img3=cv2.resize(img3,(1280,720))
        cv2.imshow("image", img3)

        cv2.waitKey(200)

        # Get the image points form the good matches
        q1 = np.float32([prev_img_kp[m.queryIdx].pt for m in good])
        q2 = np.float32([curr_img_kp[m.trainIdx].pt for m in good])
        return q1, q2

In [51]:

img_paths_list=[str(images_path+"image"+str(i)+".jpg") for i in range (1,len(os.listdir(images_path))+1) ]

for i in range(2,len(img_paths_list)):
  
  prev_img=cv2.imread(img_paths_list[i-1])
  current_img=cv2.imread(img_paths_list[i])
  
  get_matches(prev_img,current_img)

cv2.destroyAllWindows()
